In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

Nacteni souboru dat

In [2]:
books = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Denys\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

Rychly EDA

In [3]:
print(ratings.shape)
print(list(ratings.columns))

(1149780, 3)
['userID', 'ISBN', 'bookRating']


In [4]:
ratings.describe()

,userID,bookRating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   userID      1149780 non-null  int64 
 1   ISBN        1149780 non-null  object
 2   bookRating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [6]:
print(books.shape)
print(list(books.columns))

(271360, 8)
['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']


In [7]:
books.head(5)

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
books.describe()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0380538687,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/044018293X.0...,http://images.amazon.com/images/P/068825294X.0...,http://images.amazon.com/images/P/068484267X.0...
freq,1,27,632,13903,7535,2,2,2


In [9]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ISBN               271360 non-null  object
 1   bookTitle          271360 non-null  object
 2   bookAuthor         271359 non-null  object
 3   yearOfPublication  271360 non-null  object
 4   publisher          271358 non-null  object
 5   imageUrlS          271360 non-null  object
 6   imageUrlM          271360 non-null  object
 7   imageUrlL          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [10]:
print(users.shape)
print(list(users.columns))

(278858, 3)
['userID', 'Location', 'Age']


In [11]:
users.describe()

,userID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   userID    278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


1. varianta: recommendations based on correlations


Podivame se na vsechny mozne knizky, obsahujici substring "Lord of the Rings"

In [13]:
merged_lor = pd.merge(books, ratings, how = "inner", on = ["ISBN"])

In [14]:
g = pd.DataFrame(merged_lor[merged_lor["bookTitle"].str.contains("Lord of the Ring", na=False)])
g.head(2)

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL,userID,bookRating
7879,8445071408,El Senor De Los Anillos: LA Comunidad Del Anil...,J. R. R. Tolkien,2001,Minotauro,http://images.amazon.com/images/P/8445071408.0...,http://images.amazon.com/images/P/8445071408.0...,http://images.amazon.com/images/P/8445071408.0...,92,7
7880,8445071408,El Senor De Los Anillos: LA Comunidad Del Anil...,J. R. R. Tolkien,2001,Minotauro,http://images.amazon.com/images/P/8445071408.0...,http://images.amazon.com/images/P/8445071408.0...,http://images.amazon.com/images/P/8445071408.0...,53808,1


Vyfiltrujeme si unikatni ISBN a Titles pro knizky, co maji v nazvu "Lord of the Ring/Rings"

In [15]:
g.drop(["bookAuthor", "yearOfPublication", "publisher", "imageUrlS", "imageUrlM", "imageUrlL", "userID", "bookRating"], axis = 1, inplace = True)

In [16]:
g.drop_duplicates(keep = "first", inplace = True)

In [17]:
g.head()

,ISBN,bookTitle
7879,8445071408,El Senor De Los Anillos: LA Comunidad Del Anil...
7895,8445071769,El Senor De Los Anillos: Las DOS Torres (Lord ...
7905,8445071777,El Senor De Los Anillos: El Retorno Del Rey (T...
43875,0345339711,"The Two Towers (The Lord of the Rings, Part 2)"
76151,0345339703,The Fellowship of the Ring (The Lord of the Ri...


Udelame merge a souboru "books" a "ratings" dle sloupce "ISBN" odstranime zbytecne sloupce:

In [18]:
merged_ratings_books = pd.merge(books, ratings, how = "inner", on = ["ISBN"])
merged_ratings_books.drop(["bookAuthor", "yearOfPublication", "publisher", "imageUrlS", "imageUrlM", "imageUrlL"], axis = 1, inplace = True)

In [19]:
merged_ratings_books = merged_ratings_books.drop_duplicates(['userID', 'bookTitle'])

In [20]:
merged_ratings_books.head()

,ISBN,bookTitle,userID,bookRating
0,0195153448,Classical Mythology,2,0
1,0002005018,Clara Callan,8,5
2,0002005018,Clara Callan,11400,0
3,0002005018,Clara Callan,11676,8
4,0002005018,Clara Callan,41385,0


Vzhledem k obrovske velikosti datasetu, nechame pouze uzivatele, co ohodnotili aspon 100 knizek a knizky, co byly ohodnoceni aspon 300krat

In [21]:
counts_userid = merged_ratings_books['userID'].value_counts()
merged_ratings_books = merged_ratings_books[merged_ratings_books['userID'].isin(counts_userid[counts_userid >= 100].index)]
counts_raitings = merged_ratings_books['bookRating'].value_counts()
merged_ratings_books = merged_ratings_books[merged_ratings_books['bookRating'].isin(counts_raitings[counts_raitings >= 300].index)]

In [22]:
#merged_ratings_books.reset_index(drop = True).head()

Vytvorime rating matrix, kde sloupce jsou ISBN knizek, a indexy jsou id jednotlivych uzivatelu:

In [23]:
ratings_pivot = merged_ratings_books.pivot(index='userID', columns='ISBN').bookRating
userID = ratings_pivot.index
ISBN = ratings_pivot.columns
print(ratings_pivot.shape)
ratings_pivot.head()

(1654, 198188)


ISBN,0000913154,0001010565,0001046438,000104687X,0001047213,0001047647,0001047663,0001047868,0001047973,000104799X,...,B0001GDNCK,B0001GMSV2,B0001I1JII,B0001I1KOG,B0001PBXMS,B0001PIOX4,B000234N3A,B000234N76,B000234NC6,B00029DGGO
userID,,,,,,,,,,,,,,,,,,,,,
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vytvorime seznamy, obsahujici ISBN, co maji v nazvu "Lord of the Rings", a pomoci for cyklu zkusime doporucit nejake dalsi knizky pro kazdou jednotlivou knizku, ktera ma v nazvu "Lord of the Ring/Rings"

Varianta pro jednu knizku:

Cyklus pro jednu knizku/seznam knizek:

In [24]:
list_of_lor_books = ['0345339738', 
                     '0395272238','0618129022', '0345340426']

list_of_lor_3_parts = ["0345339738", "0345339703"]

for i in list_of_lor_3_parts:
    print('Looking for similarities for the following book: ' + str(merged_ratings_books[merged_ratings_books["ISBN"] == i]["bookTitle"].unique()))
    lor_ratings = ratings_pivot[i]
    similar_to_lor = ratings_pivot.corrwith(lor_ratings)
    corr_lor = pd.DataFrame(similar_to_lor, columns=['pearsonR'])
    corr_lor.dropna(inplace=True)
    corr_lor_df = pd.DataFrame(data = corr_lor)
    corr_lor_with_names = pd.merge(corr_lor_df, books, on = ['ISBN'])
    corr_lor_with_names.drop(["imageUrlS", "imageUrlM", "imageUrlL"], axis = 1, inplace = True)
    corr_lor_with_names.sort_values(by = ["pearsonR"], ascending = False)
    display(corr_lor_with_names.head())
    print("\n")

Looking for similarities for the following book: ['The Return of the King (The Lord of the Rings, Part 3)']


C:\Users\Denys\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\Denys\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,ISBN,pearsonR,bookTitle,bookAuthor,yearOfPublication,publisher
0,0007109490,1.0,The Healing Paw: Not All Angels Have Wings,Billy Roberts,2001,Thorsons Publishers
1,0007154615,1.0,Unless : A Novel,Carol Shields,2003,Perennial
2,0020434901,-1.0,"M. C. Higgins, the Great",Virginia Hamilton,1987,Simon Pulse
3,0020449313,-1.0,The Yearling (Scribner Classics),Marjorie Kinnan Rawlings,1988,Simon Pulse
4,0020868308,1.0,Mere Christianity: A revised and enlarged edit...,C. S Lewis,1984,Macmillan Pub. Co




Looking for similarities for the following book: ['The Fellowship of the Ring (The Lord of the Rings, Part 1)']


C:\Users\Denys\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\Denys\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,ISBN,pearsonR,bookTitle,bookAuthor,yearOfPublication,publisher
0,0020446500,-1.000000,Z for Zachariah,Robert C. O'Brien,1987,Simon Pulse
1,0020449313,0.409644,The Yearling (Scribner Classics),Marjorie Kinnan Rawlings,1988,Simon Pulse
2,002542730X,-0.210860,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,0028604202,-1.000000,Politically Correct Holiday Stories: For an En...,James Finn Garner,1995,John Wiley &amp; Sons Inc
4,0030617316,1.000000,"Unfortunately, She Was Also Wired for Sound (A...",G. B. Trudeau,1982,Henry Holt &amp; Co


2. varianta: Collaborative Filtering Using k-Nearest Neighbors (kNN)

Pouzijeme stejnou pivot tabulku z predchozi metody a vymenime NaNs za 0:

In [25]:
ratings_pivot.fillna(0, inplace = True)
ratings_pivot = ratings_pivot.astype(np.int32)
ratings_pivot.head()

ISBN,0000913154,0001010565,0001046438,000104687X,0001047213,0001047647,0001047663,0001047868,0001047973,000104799X,...,B0001GDNCK,B0001GMSV2,B0001I1JII,B0001I1KOG,B0001PBXMS,B0001PIOX4,B000234N3A,B000234N76,B000234NC6,B00029DGGO
userID,,,,,,,,,,,,,,,,,,,,,
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
882,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Vytvorime globalni promeny a pouzijeme funkci, ktera hleda k podobnosti a indexu pro danou knizku

In [26]:
global metric,k
k = 5
metric = 'cosine'

def findksimilaritems(item_id, ratings, metric=metric, k=k):
    similarities=[]
    indices=[]
    ratings=ratings.T
    loc = ratings.index.get_loc(item_id)
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(ratings)
    
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()

    return similarities,indices

In [27]:
#findksimilaritems("0345339703", ratings_pivot)

Update stavajici funkci: jako vstupni promennou muzeme pouzit jednu knizku/seznam knizek; jako output mame procento podobnosti a nazev podobnych knizek

In [28]:
def loop_on_findksimilaritems(list_of_isbn, ratings):
    
    trasposed_ratings = ratings.T
    
    for i in list_of_isbn:
        print("Recommendations for the following book: " + str(merged_ratings_books[merged_ratings_books["ISBN"] == i]["bookTitle"].unique()))
        print("\n")
        g =  findksimilaritems(i, ratings)
        similarities = g[0]
        indecies = [x for l in g[1] for x in l]
        
        for f, b in zip(similarities, indecies):
            print("Similarity is " + str(f) + " for book " + str(b))
            m = trasposed_ratings.index[b]
            df_r = pd.DataFrame(merged_ratings_books[merged_ratings_books['ISBN'] == m])
            print("Book name: " + str(df_r["bookTitle"].unique()))
            print("\n")
       
        print("\n")
        print("\n")

In [29]:
loop_on_findksimilaritems(list_of_lor_3_parts, ratings_pivot)

Recommendations for the following book: ['The Return of the King (The Lord of the Rings, Part 3)']


Similarity is 1.0 for book 32499
Book name: ['The Return of the King (The Lord of the Rings, Part 3)']


Similarity is 0.4071607440806918 for book 32498
Book name: ['The Two Towers (The Lord of the Rings, Part 2)']


Similarity is 0.3986177971632725 for book 32497
Book name: ['The Fellowship of the Ring (The Lord of the Rings, Part 1)']


Similarity is 0.3208557984820213 for book 97932
Book name: ["Star Wars: Tales from Jabba's Palace (Star Wars (Random House Paperback))"]


Similarity is 0.3089626875748852 for book 100504
Book name: ['Even Steven and Odd Todd (Hello Math Reader. Level 3)']


Similarity is 0.297348462766722 for book 34348
Book name: ['Crazy in Alabama']






Recommendations for the following book: ['The Fellowship of the Ring (The Lord of the Rings, Part 1)']


Similarity is 1.0 for book 32497
Book name: ['The Fellowship of the Ring (The Lord of the Rings, Part 1)']


